## Logistic Version of the Isolet Model Test in Wallaroo

The following demonstrates how to use a converted sk-learn Logistics ML model converted to ONNX  in Wallaroo.  This is meant to be a follow up to the tutorial `convert-sklearn-logistic-to-onnx.ipyb`, which demonstrates how to convert the sk-learn logicistics model to ONNX format and generates the file `isolet_logistic_model_numclass.onnx` that is used in this tutorial.

For more details on using Wallaroo workspaces, pipelines, and models, see the [Wallaroo Documentation site](https://docs.wallaroo.ai).

This tutorial includes the following assets:

* `isolet_test_data.tsv`:  A test file that can be used to verify the output of the converted logistic model.
* `test-converted-sklearn-logistics-to-onnx.ipynb`: This Jupyter Notebook that demonstrates the use of the converted sk-learn logistic ML model in ONNX with Wallaroo.

## Test Steps

### Load Libraries

The first step in the process is to load the necessary libraries.

In [1]:
# from their artifacts

import json
import wallaroo
import numpy
import pandas

from wallaroo.object import EntityNotFoundError

### Connect to Wallaroo Instance

Next we'll connect to the Wallaroo Instance with the `Client` method.

In [2]:
wl = wallaroo.Client()

### Connect to Workspace

Either create or use an existing workspace.  In this example, we will connect to the existing workspace `isolettest` by using the `set_current_workspace` command.  For more details on connecting to a workspace, see the [Wallaroo SDK Essentials guide](https://docs.wallaroo.ai/wallaroo-sdk/wallaroo-sdk-essentials-guide/).

![Wallaroo Workspace Example](images/isolet-workspace-example.png)

In [3]:
workspace_name = 'isolettest'
pipeline_name = 'isoletpipeline'
model_name = 'isolettest'

In [4]:
def get_workspace(name):
    wl = wallaroo.Client()
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    wl = wallaroo.Client()
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

isoletpipeline = get_pipeline(pipeline_name)
isoletpipeline

name,isolet-pipeline
created,2022-08-03 15:19:30.768979+00:00
last_updated,2022-08-03 15:21:09.416945+00:00
deployed,False
tags,
steps,isolet-test


### Upload Model

Upload the `isolet_logistic_model_numclass.onnx` model as `isolet-test` with the `upload_model` method:

In [5]:
model = wl.upload_model(model_name, './isolet_logistic_model_numclass.onnx').configure()

In [6]:
isoletpipeline.add_model_step(model)

name,isolet-pipeline
created,2022-08-03 15:19:30.768979+00:00
last_updated,2022-08-03 15:21:09.416945+00:00
deployed,False
tags,
steps,isolet-test


### Deploy Pipeline

Pipelines can be deployed either with the Wallaroo Dashboard user interface or through the SDK.  For this example, we have the pipeline `isolet-pipeline` deployed and assigned to the variable `isoletpipeline`.  For more details, see the [Wallaroo Pipeline Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-pipeline-management/).

![Wallaroo Workspace Example](images/isolet-pipeline-example.png)

We can check the status of the pipeline with the Wallaroo pipeline `status` method.

In [7]:
isoletpipeline.deploy()

Waiting for deployment - this will take up to 45s .... ok


name,isolet-pipeline
created,2022-08-03 15:19:30.768979+00:00
last_updated,2022-08-03 15:26:20.643739+00:00
deployed,True
tags,
steps,isolet-test


### Load the Data

Load the data from the file `isolet_test_data.tsv` into the variable `testdata`, and turn it into a data dictionary variable labeled `input_dict`.

In [8]:
# read in data
testdata = pandas.read_csv('isolet_test_data.tsv', sep='\t')

input_dict = {
    'tensor': testdata.values.tolist()
}

We'll display the data shape to verify it has 9 rows and 617 columsn per row.

In [9]:
# report the size of the data  -  9 rows, 617 columns each
testdata.shape

(9, 617)

### Run Inference

Use the Wallaroo pipeline `infer` method to run an inference and store the results into the variable `result`, then display the data from the result.

In [10]:
result = isoletpipeline.infer(input_dict)

Waiting for inference response - this will take up to 45s ....... ok


In [11]:
result[0].raw['outputs']

[{'Int64': {'v': 1, 'dim': [9], 'data': [0, 0, 1, 1, 0, 0, 1, 1, 0]}},
 {'Float': {'v': 1,
   'dim': [9, 2],
   'data': [0.9999629855155945,
    3.701448440551758e-05,
    0.9977498650550842,
    0.0022501349449157715,
    0.004735887050628662,
    0.9952641129493713,
    0.03107866644859314,
    0.9689213037490845,
    0.9996306896209717,
    0.0003693103790283203,
    0.9992027282714844,
    0.000797271728515625,
    0.14781257510185242,
    0.8521873950958252,
    0.08045655488967896,
    0.919543445110321,
    0.9987874031066895,
    0.0012125670909881592]}}]

### Closing

Once the tests are done, the pipeline should be undeployed to allocate the resources back to the Wallaroo instance.  For more details, see the [Wallaroo Pipeline Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-pipeline-management/).

In [12]:
isoletpipeline.undeploy()

Waiting for undeployment - this will take up to 45s .............................. ok


name,isolet-pipeline
created,2022-08-03 15:19:30.768979+00:00
last_updated,2022-08-03 15:26:20.643739+00:00
deployed,False
tags,
steps,isolet-test
